# Part 5 - Explore the Clusters
In this notebook, I will explore the clusters created by KMeans and create an interactive Folium map of the clusters.

In [3]:
#import libraries
import requests
import numpy as np
import pandas as pd
import ibm_boto3
import io

In [2]:
#!conda install -c conda-forge folium=0.5.0 --yes

In [4]:
from IPython.display import Image 
from IPython.core.display import HTML 
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib as mpl

In [5]:
# The code was removed by Watson Studio for sharing.

In [6]:
#Import dataframe
obj = resource.Object(bucket_name=bucket, key='VenueDataWithClusters.json').get()
df = pd.read_json(io.BytesIO(obj['Body'].read()))
df.sort_index(inplace=True)
df.head()

,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
0,Tokyo,Japan,Asia,35.682839,139.759455,Café,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Lounge,Garden,French Restaurant,Dessert Shop,Thai Restaurant,Steakhouse,0
1,Delhi,India,Asia,28.651718,77.221939,Indian Restaurant,Bar,Café,Flea Market,Lounge,Asian Restaurant,Ice Cream Shop,South Indian Restaurant,Portuguese Restaurant,Plaza,3
2,Mexico City,Mexico,North America,19.432601,-99.133342,Mexican Restaurant,Art Museum,Museum,Arts & Crafts Store,Bar,Ice Cream Shop,Boutique,Bakery,Restaurant,Café,11
3,São Paulo,Brazil,South America,-23.550651,-46.633382,Japanese Restaurant,Brazilian Restaurant,Café,Bakery,Snack Place,Bookstore,Plaza,Cosmetics Shop,Chinese Restaurant,Art Gallery,2
4,Lagos,Nigeria,Africa,6.455057,3.394179,Lounge,African Restaurant,Bar,Café,Pizza Place,Market,Art Gallery,Italian Restaurant,Hotel,Shopping Mall,11


In [7]:
#Look at the metro areas with cluster labels
df.loc[:,['Metropolitan','Country','Cluster Labels']].head()

,Metropolitan,Country,Cluster Labels
0,Tokyo,Japan,0
1,Delhi,India,3
2,Mexico City,Mexico,11
3,São Paulo,Brazil,2
4,Lagos,Nigeria,11


In [8]:
#How many metro areas are in each cluster?
df_count = df.loc[:,['Metropolitan','Cluster Labels']].groupby('Cluster Labels').count()
df_count

,Metropolitan
Cluster Labels,
0,17
1,13
2,32
3,8
4,1
5,1
6,1
7,1
8,1


In [9]:
#Print out the interesting clusters, clusters where n > 2
cluster_list=[]
c=0
for i in df_count[df_count['Metropolitan']>2].index:
    cluster_list.append(df[df['Cluster Labels']==i].index)
    print('Cluster ',i)
    with pd.option_context('display.max_rows', None):
        display(df.iloc[cluster_list[c],:])
    c=c+1

Cluster  0


,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
0,Tokyo,Japan,Asia,35.682839,139.759455,Café,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Lounge,Garden,French Restaurant,Dessert Shop,Thai Restaurant,Steakhouse,0
6,Cairo,Egypt,Africa,30.048819,31.243666,Café,Lounge,Middle Eastern Restaurant,Italian Restaurant,Theater,Egyptian Restaurant,Lebanese Restaurant,Coffee Shop,Hotel Bar,Boat or Ferry,0
7,Kyoto,Japan,Asia,35.021041,135.755607,Café,Japanese Restaurant,Bakery,Sake Bar,Dessert Shop,Palace,Ramen Restaurant,Soba Restaurant,Coffee Shop,Donburi Restaurant,0
11,Dhaka,Bangladesh,Asia,23.759357,90.378814,Café,Shopping Mall,Fast Food Restaurant,Restaurant,Art Gallery,Asian Restaurant,Ice Cream Shop,Burger Joint,Department Store,Electronics Store,0
14,Karachi,Pakistan,Asia,25.144690,67.184777,BBQ Joint,Bakery,Café,Pakistani Restaurant,Asian Restaurant,Frozen Yogurt Shop,Ice Cream Shop,Indian Restaurant,Market,Mobile Phone Shop,0
18,Bangkok,Thailand,Asia,13.753893,100.816080,Thai Restaurant,Noodle House,Asian Restaurant,Coffee Shop,Convenience Store,Flea Market,Shopping Mall,Café,Market,Szechuan Restaurant,0
19,Kolkata,India,Asia,22.567746,88.347602,Café,Indian Restaurant,Mughlai Restaurant,History Museum,Nightclub,Park,Indian Sweet Shop,Multiplex,Pool,Bookstore,0
20,Tehran,Iran,Asia,35.700618,51.401378,Café,Bookstore,Theater,Persian Restaurant,Coffee Shop,Breakfast Spot,Art Gallery,Hookah Bar,Fast Food Restaurant,Plaza,0
30,Kinshasa,Democratic Republic of the Congo,Africa,-4.321706,15.312597,Café,Restaurant,Fast Food Restaurant,Lounge,Bakery,Steakhouse,Pool,Plaza,Pizza Place,Furniture / Home Store,0
44,Johannesburg,South Africa,Africa,-26.205000,28.049722,Coffee Shop,Café,Art Gallery,Nightclub,Flea Market,Asian Restaurant,Bar,Theater,Portuguese Restaurant,Indian Restaurant,0


Cluster  1


,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
8,Wuhan,China,Asia,30.595105,114.299935,Shopping Mall,Park,Chinese Restaurant,Department Store,Asian Restaurant,Beer Garden,Coffee Shop,Hubei Restaurant,Movie Theater,Pedestrian Plaza,1
10,Chengdu,China,Asia,30.662420,104.063322,Chinese Restaurant,Noodle House,Department Store,Tea Room,Szechuan Restaurant,Park,Pedestrian Plaza,Shopping Mall,Coffee Shop,Bookstore,1
12,Shanghai,China,Asia,31.232274,121.469175,Chinese Restaurant,Lounge,Hotel Bar,Coffee Shop,Japanese Restaurant,Art Museum,Gym,Café,Multiplex,Park,1
13,Chongqing,China,Asia,29.558571,106.549282,Shopping Mall,Coffee Shop,Park,Chinese Restaurant,Department Store,Electronics Store,German Restaurant,Nightclub,Bar,Scenic Lookout,1
22,Hangzhou,China,Asia,30.248963,120.205234,Shopping Mall,Chinese Restaurant,Coffee Shop,Dessert Shop,Plaza,Movie Theater,Asian Restaurant,New American Restaurant,Steakhouse,Sporting Goods Shop,1
24,Surabaya,Indonesia,Asia,-7.245972,112.737827,Food Truck,Chinese Restaurant,Camera Store,Indonesian Restaurant,Bookstore,Coffee Shop,Flea Market,Food Court,Fried Chicken Joint,History Museum,1
33,Nanjing,China,Asia,32.060974,118.791646,Shopping Mall,Nightclub,Café,Park,Indian Restaurant,Multiplex,Electronics Store,Japanese Restaurant,Chinese Restaurant,Mountain,1
37,Jinan,China,Asia,36.650701,117.114004,Shopping Mall,Park,Coffee Shop,Fast Food Restaurant,Department Store,Electronics Store,Gourmet Shop,Hotel Bar,Italian Restaurant,Movie Theater,1
39,Harbin,China,Asia,45.765667,126.616058,Shopping Mall,Department Store,Park,Eastern European Restaurant,Asian Restaurant,Bar,Café,Coffee Shop,Nightclub,Plaza,1
45,Guangzhou,China,Asia,23.130196,113.259294,Chinese Restaurant,Shopping Mall,Nightclub,History Museum,Park,Movie Theater,Asian Restaurant,Coffee Shop,Cantonese Restaurant,Café,1


Cluster  2


,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
3,São Paulo,Brazil,South America,-23.550651,-46.633382,Japanese Restaurant,Brazilian Restaurant,Café,Bakery,Snack Place,Bookstore,Plaza,Cosmetics Shop,Chinese Restaurant,Art Gallery,2
15,Moscow,Russia,Europe,55.750446,37.617494,Boutique,Plaza,Italian Restaurant,Coffee Shop,History Museum,Fountain,Art Museum,Bakery,Park,Jewelry Store,2
17,Istanbul,Turkey,Europe,41.009633,28.965165,Turkish Restaurant,Jewelry Store,Kebab Restaurant,Café,Restaurant,Garden,Art Gallery,Hotel,Seafood Restaurant,Coffee Shop,2
21,London,United Kingdom,Europe,51.507322,-0.127647,Theater,Plaza,Garden,Coffee Shop,Wine Bar,Dessert Shop,Italian Restaurant,Pizza Place,Ice Cream Shop,Pub,2
23,Jakarta,Indonesia,Asia,-6.175394,106.827183,Park,Indonesian Restaurant,Asian Restaurant,Noodle House,Soup Place,Soccer Field,Skate Park,Plaza,Performing Arts Venue,Museum,2
27,Paris,France,Europe,48.856610,2.351499,French Restaurant,Ice Cream Shop,Art Gallery,Wine Bar,Bookstore,Burger Joint,Park,Plaza,Coffee Shop,Theater,2
34,Seoul,South Korea,Asia,37.566679,126.978291,Chinese Restaurant,Korean Restaurant,Coffee Shop,Japanese Restaurant,Café,Hotel Bar,Bakery,Plaza,Bookstore,Lounge,2
35,Rhine-Ruhr,Germany,Europe,51.437302,7.334511,Furniture / Home Store,Bakery,Café,Ice Cream Shop,German Restaurant,Big Box Store,Doner Restaurant,Supermarket,Arts & Crafts Store,Restaurant,2
36,Lahore,Pakistan,Asia,31.565608,74.314177,Pakistani Restaurant,Fast Food Restaurant,Ice Cream Shop,Restaurant,Market,Department Store,Italian Restaurant,BBQ Joint,Café,Breakfast Spot,2
40,Chicago,United States,North America,41.875562,-87.624421,Coffee Shop,Pizza Place,Music Venue,Pub,Italian Restaurant,Gym / Fitness Center,Dance Studio,Museum,Plaza,Public Art,2


Cluster  3


,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
1,Delhi,India,Asia,28.651718,77.221939,Indian Restaurant,Bar,Café,Flea Market,Lounge,Asian Restaurant,Ice Cream Shop,South Indian Restaurant,Portuguese Restaurant,Plaza,3
5,Mumbai,India,Asia,18.938771,72.835335,Indian Restaurant,Café,Bakery,Seafood Restaurant,Bar,Chinese Restaurant,Ice Cream Shop,Fast Food Restaurant,Coffee Shop,Pizza Place,3
28,Bangalore,India,Asia,12.979120,77.591300,Indian Restaurant,Lounge,Italian Restaurant,Ice Cream Shop,Breakfast Spot,Chinese Restaurant,Coffee Shop,Japanese Restaurant,Park,French Restaurant,3
38,Chennai,India,Asia,13.080172,80.283833,Indian Restaurant,Multiplex,Donut Shop,Museum,Bookstore,Clothing Store,Department Store,Sandwich Place,Electronics Store,Farmers Market,3
55,Hyderabad,India,Asia,17.388786,78.461065,Indian Restaurant,Multiplex,Bakery,Chinese Restaurant,Dessert Shop,South Indian Restaurant,Ice Cream Shop,Hotel Bar,Vegetarian / Vegan Restaurant,Café,3
60,Pune,India,Asia,18.520306,73.854319,Indian Restaurant,Vegetarian / Vegan Restaurant,Snack Place,Restaurant,Fast Food Restaurant,Dessert Shop,Ice Cream Shop,Café,Sandwich Place,Seafood Restaurant,3
74,Ahmedabad,India,Asia,23.021624,72.579707,Indian Restaurant,Fast Food Restaurant,Tea Room,Café,Ice Cream Shop,Snack Place,Coffee Shop,Dessert Shop,Shopping Mall,Mexican Restaurant,3
87,Dubai-Sharjah-Ajman,United Arab Emirates,Asia,25.235625,55.296689,Indian Restaurant,Middle Eastern Restaurant,Café,Restaurant,Seafood Restaurant,Asian Restaurant,Korean Restaurant,Park,Hookah Bar,Filipino Restaurant,3


Cluster  11


,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
2,Mexico City,Mexico,North America,19.432601,-99.133342,Mexican Restaurant,Art Museum,Museum,Arts & Crafts Store,Bar,Ice Cream Shop,Boutique,Bakery,Restaurant,Café,11
4,Lagos,Nigeria,Africa,6.455057,3.394179,Lounge,African Restaurant,Bar,Café,Pizza Place,Market,Art Gallery,Italian Restaurant,Hotel,Shopping Mall,11
9,Los Angeles,United States,North America,34.053691,-118.242767,Bar,Coffee Shop,Sushi Restaurant,Plaza,Mexican Restaurant,Ice Cream Shop,Ramen Restaurant,Sandwich Place,Seafood Restaurant,Mediterranean Restaurant,11
25,Buenos Aires,Argentina,South America,-34.607562,-58.437076,Argentinian Restaurant,Gym,Pizza Place,Bakery,Ice Cream Shop,Café,Restaurant,Burger Joint,Empanada Restaurant,Bar,11
31,Rio de Janeiro,Brazil,South America,-22.911014,-43.209373,Brazilian Restaurant,Bar,Restaurant,Bakery,Gym / Fitness Center,Samba School,Café,Coffee Shop,Gym,Pharmacy,11
41,Lima,Peru,South America,-12.062107,-77.036526,Park,Seafood Restaurant,Bar,Peruvian Restaurant,Shopping Mall,Nightclub,Electronics Store,Plaza,Pub,Restaurant,11
48,Chūkyō (Nagoya),Japan,Asia,35.138700,136.966392,Ramen Restaurant,Park,Café,Bakery,Restaurant,Yoshoku Restaurant,Chinese Restaurant,Udon Restaurant,Zoo Exhibit,Grocery Store,11
51,Randstad,Netherlands,Europe,52.378042,5.226072,Sushi Restaurant,Restaurant,Supermarket,Gym / Fitness Center,Shopping Mall,Gym,Grocery Store,Electronics Store,Park,Pet Store,11
52,Busan,South Korea,Asia,35.179953,129.075236,Supermarket,Fast Food Restaurant,BBQ Joint,Concert Hall,Gukbap Restaurant,Korean Restaurant,Multiplex,Noodle House,Outdoor Sculpture,Park,11
54,Dallas-Fort Worth,United States,North America,32.759907,-97.173563,Ice Cream Shop,American Restaurant,BBQ Joint,Bar,Park,Mexican Restaurant,Liquor Store,Grocery Store,Golf Course,Supermarket,11


In [ ]:
#Take a look at the other clusters, those with 1 or 2 metro areas
other_list=[]
c=0
for i in df_count[df_count['Metropolitan']<=2].index:
    other_list.append(df[df['Cluster Labels']==i].index)
    print('Cluster ',i)
    with pd.option_context('display.max_rows', None):
        display(df.iloc[other_list[c],:])
    c=c+1

In [ ]:
#Create an interactive folium map with different color markers for each cluster
map_clusters = folium.Map(location=[0,0], zoom_start=2)

#set color scheme for the clusters
map_colors = [colors.rgb2hex(cm.tab20b(i)) for i in range(15)]

# add markers to the map
markers_colors = []
for latitude, longitude, metro, country, cluster in zip(df['Latitude'], df['Longitude'], df['Metropolitan'], df['Country'], df['Cluster Labels']):
    label = folium.Popup(str(metro) + ', ' + str(country)+'- Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color=map_colors[cluster-1],
        fill=True,
        fill_color=map_colors[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters.save('map.html')
map_clusters

### View the interactive map
https://kathryndh.github.io/clustermap.html

In [ ]:
#Save the map as an html file
obj = io.open('map.html','rb')
resource.Bucket(name=bucket).put_object(Key='map.html', Body=obj.read())
obj.close()
print('Saved')